In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("ColabSparkSession") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/16 14:24:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# <font color="blue">Understanding Spark partition </font>

In [2]:
Run below command to get block information about the file

hdfs fsck /user/training/spark-data/rating-small/ -blocks

SyntaxError: invalid syntax (2840780527.py, line 1)

### Create a Dataframe

In [3]:
smallratingDF = spark.read.parquet("spark-data/rating-small")

25/07/16 14:25:03 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: spark-data/rating-small.
java.io.FileNotFoundException: File spark-data/rating-small does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.Resol

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/shivam/Downloads/Spark/Spark_Optimization_new/Day1/spark-data/rating-small. SQLSTATE: 42K03

In [2]:
smallratingDF.rdd.getNumPartitions()

22

__Spark decided to read these small 601 files in 22 partition__

In [4]:
hugeratingDF = spark.read.parquet("spark-data/rating-huge")

In [5]:
hugeratingDF.rdd.getNumPartitions()

4

__Spark decided to read these small 1 files in 2 partition__

In [5]:
ratingDF = spark.read.option("header","true").csv("spark-data/ratings.csv")
ratingDF.rdd.getNumPartitions()

6

In [8]:
spark.conf.set("spark.sql.adaptive.enabled","true")
spark.conf.set("spark.sql.shuffle.partitions",200)
ratingDF.groupBy("rating").count().write.mode("overwrite").csv("spark-data/output/ratings")

__This time spark decided to read this 646MB file in 6 partitions__

<font color="sky blue">Question: How Spark deciding on number of partition? Is it random or based on some formula?</font>

 Calculation of Spark Partition depends on below properties:
        
- spark.default.parallelism (default: Total No. of CPU cores)
- spark.sql.files.maxPartitionBytes (default: 128 MB)
- spark.sql.files.openCostInBytes (default: 4 MB)


First it calculates bytePercore which is based on 

- sum of size of all files
- no of files
- total no of cores of executors (default.parallelism)
- spark.sql.files.openCostInBytes (default: 4 MB)

<font color="light blue">___bytesPerCore = (Sum of sizes of all data files + No. of files * openCostInBytes) / default.parallelism___</font>


Then we compare above calculated value with spark.sql.files.maxPartitionBytes (default: 128 MB),
whichever is minimum that becomes our partition size

<font color="light blue">maxSplitBytes = Minimum(maxPartitionBytes, bytesPerCore)</font>



### <font color="blue"> Assignment </font>

In How many Spark Partitions, below files will be read

54 parquet files, 63 MB each, all 3 config parameters at default, No. of core equal to 10

54 parquet files, 40 MB each, all 3 config parameters at default, No. of core equal to 10


54 parquet files, 40 MB each, maxPartitionBytes set to 88 MB, other two configs at default values., No. of core equal to 10